In [1]:
import numpy as np
import pandas as pd 
from glob import glob
from os import path
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from torch.autograd import Variable
import torch.utils.data as utils
from time import sleep
import tensorflow as tf

c:\users\shaur\appdata\local\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [31]:
N = 1200
batch_size = 32
seq_len = 20
num_features = 1
window_size = seq_len

In [32]:
def generate_data(N, seq_len):
    N_train = int(N*0.8)
    X = np.zeros((N, seq_len), dtype=np.float32)
    y = np.zeros((N,1), dtype=np.float32)
    
    indices = np.random.randint(0, N, size=N//2)
    
    X[indices, 0] = 1.0
    y[indices, 0] = 1.0
    
    X_train, y_train = X[:N_train], y[:N_train]
    X_test, y_test = X[N_train:], y[N_train:]
    
    return X_train, y_train, X_test, y_test
    

In [33]:
def generate_synthetic_test_data(N, seq_len):
    N_train = int(N*0.8)
    X = np.reshape(np.arange(0, N*seq_len, dtype=np.float32),(N, seq_len))
    y = np.reshape(np.arange(0,N, dtype=np.float32), (N,1))
    
    
    X_train, y_train = X[:N_train], y[:N_train]
    X_test, y_test = X[N_train:], y[N_train:]
    
    return X_train, y_train, X_test, y_test

In [34]:
X_train, y_train, X_test, y_test = generate_data(N, seq_len)

In [35]:
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        N = X.shape[0]        
        self.N = N
        
    def __len__(self):
        return self.N
    
    def __getitem__(self, index):
        x = np.expand_dims(self.X[index], -1)
        y = self.y[index]
        return x,y

In [36]:
class Net(nn.Module):
    def __init__(self, num_features, num_hidden, num_lstm_layers, batch_size,
                seq_len, window_size, step=1):
        super(Net, self).__init__()        
        self.num_hidden = num_hidden
        self.num_lstm_layers = num_lstm_layers
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.window_size = window_size
        self.step = 1
        self.lstm1 = nn.LSTM(input_size=1, hidden_size=num_hidden, 
                             num_layers=num_lstm_layers, batch_first=True)
        self.fc1 = nn.Linear(num_hidden, 1)
        self.sigmoid = nn.Sigmoid()
        self.hidden = list()
        
        
        
    def init_hidden(self):
        self.hidden = [(Variable(torch.zeros(self.num_lstm_layers, self.batch_size, self.num_hidden, dtype=torch.float32)), 
                       Variable(torch.zeros(self.num_lstm_layers, self.batch_size, self.num_hidden, dtype=torch.float32)))]
    
    def forward(self, batch_data):
        num_steps = self.seq_len - self.window_size + 1
        w = self.window_size
        for i in range(num_steps):            
            x = batch_data[:,i:i+w]        
            lstm_out, h = self.lstm1(x, self.hidden[-1])
            self.hidden.append(h)
            
        
        y_pred = self.sigmoid(self.fc1(lstm_out[:,-1]))
        return y_pred

In [37]:
num_epochs = 15
train_ds = SequenceDataset(X_train, y_train)
train_dataloader = utils.DataLoader(train_ds, batch_size=batch_size, drop_last=True)
net = Net(num_features=1, num_hidden=64, num_lstm_layers=1, batch_size=batch_size,
         seq_len=seq_len, window_size=window_size)
learning_rate = 1e-3
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

test_ds = SequenceDataset(X_test, y_test)
test_dataloader = utils.DataLoader(test_ds, batch_size=batch_size, drop_last=True)

In [38]:
for epoch in range(num_epochs):
    tr_acc = 0
    tr_loss = 0
    total = 0
    for batch_data, batch_labels in train_dataloader:
        net.init_hidden()        
        optimizer.zero_grad()
        y_pred = net(batch_data)
        loss = loss_fn(y_pred, batch_labels)
        tr_loss += loss.item()
        mean_tr_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
        total += batch_size
        
        y_pred = (y_pred > 0.5).type(torch.FloatTensor)
        accuracy = (y_pred == batch_labels).sum().item()
        tr_acc += accuracy
        
        
    tr_loss = tr_loss/total
    tr_acc = tr_acc/total
        
    test_loss = 0
    test_acc = 0
    total = 0
    for batch_data, batch_labels in test_dataloader:
        net.init_hidden()
        y_pred = net(batch_data)
        loss = loss_fn(y_pred, batch_labels)       
        test_loss += loss.item()
        
        y_pred = (y_pred > 0.5).type(torch.FloatTensor)
        accuracy = (y_pred == batch_labels).sum().item()
        test_acc += accuracy
        
        total += batch_size
    
    test_loss = test_loss/total
    test_acc = test_acc/total
    print("Epoch ", epoch)
    print("Training loss: ", tr_loss, ", Training accuracy: ", tr_acc)
    print("Test loss: ", test_loss, ", Test accuracy: ", test_acc)




Epoch  0
Training loss:  0.021014215238392354 , Training accuracy:  0.6114583333333333
Test loss:  0.020621426669614657 , Test accuracy:  0.6294642857142857
Epoch  1
Training loss:  0.020915064339836437 , Training accuracy:  0.6114583333333333
Test loss:  0.020664540784699575 , Test accuracy:  0.6294642857142857
Epoch  2
Training loss:  0.020886693087716898 , Training accuracy:  0.6114583333333333
Test loss:  0.02062721018280302 , Test accuracy:  0.6294642857142857
Epoch  3
Training loss:  0.020887828307847182 , Training accuracy:  0.6114583333333333
Test loss:  0.020625903936369077 , Test accuracy:  0.6294642857142857
Epoch  4
Training loss:  0.020163585276653368 , Training accuracy:  0.6114583333333333
Test loss:  0.012450687055076872 , Test accuracy:  0.6294642857142857
Epoch  5
Training loss:  0.0064618337356174985 , Training accuracy:  0.940625
Test loss:  0.0019450946544696177 , Test accuracy:  1.0
Epoch  6
Training loss:  0.0014286195548872153 , Training accuracy:  1.0
Test loss

In [39]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=64, stateful=True, batch_input_shape=(batch_size, window_size, num_features)))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])


Instructions for updating:
Colocations handled automatically by placer.


In [175]:
train_ds = SequenceDataset(X_train, y_train)
train_dataloader = utils.DataLoader(ds, batch_size=batch_size, drop_last=True)
for epoch in range(num_epochs):
    
    for batch_data, batch_label in train_dataloader:
        
        model.reset_states()

        batch_data, batch_label = ds.get_batch()            
        loss, acc = model.train_on_batch(batch_data, batch_label)
        mean_tr_loss.append(loss)
        mean_tr_acc.append(acc)
    print('Training loss {}, Training accuracy : {}'.format(np.mean(mean_tr_loss), np.mean(mean_tr_acc)))

        

Training loss 0.6696522831916809, Training accuracy : 0.6104166507720947
Training loss 0.37252891063690186, Training accuracy : 0.8138889074325562
Training loss 0.002142030280083418, Training accuracy : 1.0
Training loss 0.0007593360496684909, Training accuracy : 1.0
Training loss 0.0004327250935602933, Training accuracy : 1.0
Training loss 0.0002880175889004022, Training accuracy : 1.0
Training loss 0.0002076311211567372, Training accuracy : 1.0
Training loss 0.00015851126227062196, Training accuracy : 1.0
Training loss 0.00012558502203319222, Training accuracy : 1.0


KeyboardInterrupt: 

In [165]:
X_test_ = X_test[:224]
y_test_ = y_test[:224]

In [169]:
test_ds = StatefulSequenceDataset(X_test_, y_test_, batch_size=32,seq_len=seq_len, window_size=window_size)

In [170]:
xx, yy = test_ds.get_batch()

In [172]:
model.reset_states()
model.predict_on_batch(np.expand_dims(xx,-1))

array([[9.9985719e-01],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [9.9985719e-01],
       [7.7694574e-05],
       [7.7694574e-05],
       [9.9985719e-01],
       [7.7694574e-05],
       [9.9985719e-01],
       [7.7694574e-05],
       [9.9985719e-01],
       [9.9985719e-01],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [9.9985719e-01],
       [7.7694574e-05],
       [7.7694574e-05],
       [9.9985719e-01],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [7.7694574e-05],
       [9.9985719e-01],
       [7.7694574e-05],
       [7.7694574e-05]], dtype=float32)

In [173]:
yy

array([[1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.]], dtype=float32)

In [15]:
net.init_hidden()

In [16]:
net(xx.unsqueeze(-1))

tensor([[0.0002],
        [0.0002],
        [0.9995],
        [0.0002],
        [0.9995],
        [0.9995],
        [0.0002],
        [0.0002],
        [0.0002],
        [0.0002],
        [0.9995],
        [0.9995],
        [0.9995],
        [0.9995],
        [0.9995],
        [0.0002],
        [0.9995],
        [0.0002],
        [0.0002],
        [0.9995],
        [0.9995],
        [0.0002],
        [0.9995],
        [0.9995],
        [0.0002],
        [0.0002],
        [0.0002],
        [0.0002],
        [0.0002],
        [0.9995],
        [0.0002],
        [0.0002]], grad_fn=<SigmoidBackward>)

In [17]:
yy

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.]])